# MCAS PS Module - Import Module and Authenticate

## Import [Unofficial] MCAS Powershell module from Github Repository
[MCAS Powershell Module [Unofficial] - Github Repo](https://github.com/microsoft/MCAS)

In [ ]:
$host.UI.RawUI.BufferSize = [System.Management.Automation.Host.Size]::new(200, 50) ## Corrects the output
Install-Module MCAS
#Import-Module MCAS 

# Grabbing commands available
Get-Command -module "MCAS" | Select-Object -Property Name | Format-Wide -Column 4
Out-Display -input "Type <font color=Yellow>'Get-Help &lt;command&gt; -Examples</font>' to get help on any modules"

## Configure Variables and store credentials in XML file

In [ ]:
$tenantUri = "domain.us3.portal.cloudappsecurity.com"

In [ ]:
$tenantName = ($tenantUri -replace "https://" | % {$_ -match "(.*).portal.cloudappsecurity.com"})
if ($Matches[1]) {$tenantName = $Matches[1]} 

$path = New-Item -ItemType Directory -Force -Path ".\Credentials"
$credentialPath = ".\$($path.name)\$tenantName.credential"
$credentialPath

## Import Credentials

In [ ]:
## Import MCAS Credential or add credential if does not exist
if( (Test-Path $credentialPath -PathType Leaf) ) {
    $CASCredential = Import-Clixml $credentialPath -ErrorAction SilentlyContinue
} else {
    Get-MCASCredential -tenantUri $tenantUri -PassThru | Export-CliXml ".\Credentials\$($Matches[1]).credential" -Force
    Write-Host "Stored credentials in $credentialPath"
}

## Query MCAS REST API and determine if credentials exists

In [ ]:
$MCASPolicies = Get-MCASPolicy
Write-Host -Foreground green "`n $($MCASPolicies.count) MCAS Policies Retrieved`n"

$MCASPolicies | Group-Object -Property policyType -NoElement | Sort-Object -Property Count -Descending | Select-Object Name,Count

## Pull all the Session policies

In [ ]:
$filterPolicy = "SESSION"
$MCASPolicies | Where-Object {$_.policyType -eq $filterPolicy} | Select-Object name | FT

In [ ]:
$filterPolicyName = "*DEMO*"
$MCASPolicies | Where-Object {$_.name -like $filterPolicyName} | FL